## Importando o dataset e quebrando as fontes em dataframes distintos

In [1]:
import pandas as pd
import numpy as np
from zipfile import ZipFile

with ZipFile('../data/tabelas.zip') as zip:
    with zip.open('tEstabelecimentos_sample.csv') as estabelecimentos:
        df_estabelecimentos = pd.read_csv(estabelecimentos)
    with zip.open('tPedidos_sample.csv') as pedidos:
        df_pedidos = pd.read_csv(pedidos)
    with zip.open('tSource_sample.csv') as source:
        df_source = pd.read_csv(source)
    with zip.open('tVendedor_sample.csv') as vendedor:
        df_vendedor = pd.read_csv(vendedor)

## Fazendo os joins entre os dataframes para uma única fonte de dados

In [2]:


# Realiza o join (inner) entre a dataset de Estabelecimento e Fonte
df_estabelecimentos = pd.merge(df_estabelecimentos.set_index("Source_id"), 
                       df_source.set_index("id"),
                       left_index = True,
                       right_index = True)

# Realiza o join (inner) entre o dataset de Estabelecimento e Vendedor
df_estabelecimentos = pd.merge(df_estabelecimentos.set_index("Vendedor_id"), 
                           df_vendedor.set_index("id"),
                           left_index = True,
                           right_index = True)

# Renomeando as colunas para facilitar o entendimento e visualização
df_pedidos = df_pedidos.rename(columns = 
                                { 
                                    "establishment_id": "id_estabelecimento",
                                    "delivery_way": "tp_entrega",
                                    "payment_type": "tp_pagamento",
                                    "enviado_em": "dt_envio"}
                                )

df_estabelecimentos = df_estabelecimentos.rename(
                            columns = 
                                { 
                                  "name": "nm_estabelecimento",
                                  "created_at": "dt_criacao",
                                  "lic": "cd_sistema",
                                  "ProdutosCadastrados": "qt_produtos_cadastrados",
                                  "city": "nm_cidade",
                                  "state": "cd_estado",
                                  "segment": "tp_segmento",
                                  "Source": "nm_canal_midia",
                                  "Vendedor": "nm_vendedor"
                                }
                          )

## JOINS entre PEDIDOS e ESTABELECIMENTOS para obter as colunas necessárias

In [3]:
df_pedidos = pd.merge(df_pedidos,
                     df_estabelecimentos[["id", "nm_estabelecimento", "cd_estado", 
                                          "nm_cidade", "cd_sistema", "nm_canal_midia", "nm_vendedor", "tp_segmento"]],
                     left_on = "id_estabelecimento",
                     right_on = "id",
                     how="inner").drop(columns={"id_y"}).rename(columns={"id_x": "id"})

df_pedidos

,id,id_estabelecimento,tp_entrega,tp_pagamento,dt_envio,nm_estabelecimento,cd_estado,nm_cidade,cd_sistema,nm_canal_midia,nm_vendedor,tp_segmento
0,1,3072,Retirada,Dinheiro,2020-03-31 18:50:36.368332,Zuppa,SP,São Paulo,t0,Linkedin,John,Japonês
1,66,3072,Retirada,Dinheiro,2020-04-03 16:32:06.830540,Zuppa,SP,São Paulo,t0,Linkedin,John,Japonês
2,2613,3072,Retirar,Dinheiro,2020-05-06 13:06:07.485394,Zuppa,SP,São Paulo,t0,Linkedin,John,Japonês
3,2760,3072,Retirar,Dinheiro,2020-05-07 13:19:56.687562,Zuppa,SP,São Paulo,t0,Linkedin,John,Japonês
4,3898,3072,Retirar,Dinheiro,2020-05-15 10:06:48.478495,Zuppa,SP,São Paulo,t0,Linkedin,John,Japonês
...,...,...,...,...,...,...,...,...,...,...,...,...
4458,4447,16184,Entrega,Crédito,2020-05-18 12:25:04.892878,Cantina da tia Cintia,SP,Sorocaba,t0,Sites de Parceiros,Lurdes,Empório
4459,4455,16184,Entrega,Débito,2020-05-18 14:09:18.677851,Cantina da tia Cintia,SP,Sorocaba,t0,Sites de Parceiros,Lurdes,Empório
4460,4098,16491,Entrega,Dinheiro,2020-05-15 22:45:50.413891,Caldos e Sopas,SP,Campinas,t2,Instagram,Marta,Café/Doceria/Encomenda
4461,4402,17084,Entrega,Débito,2020-05-17 23:14:03.695379,CarLF,SP,São Paulo,t1,Facebook,John,Restaurante Tipo A


## [VISUALIZAÇÃO] SEGMENTO

In [221]:
import pandas as pd
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

df_ped = pd.DataFrame(df_pedidos.groupby(["tp_segmento", "nm_estabelecimento"]).count()["id"])
df_ped = df_ped.reset_index()

qt_pedidos = go.Scatter(x = df_ped["tp_segmento"],
                        y = df_ped["id"],
                        text = df_ped["nm_estabelecimento"],
                        name = 'Qtd. Pedidos realizados',
                        mode = 'markers',
                        marker =  {
                                    'color' : '#0c2461',
                                    'line' : {'width': 1, 'color': '#78e08f'}
                                  },
                        opacity=.9)

# Gráfico por segmento
qt_produtos = go.Scatter(x = df_estabelecimentos['tp_segmento'],
                         y = df_estabelecimentos['qt_produtos_cadastrados'],
                         text = df_estabelecimentos["nm_estabelecimento"],
                         mode = 'markers',
                         name = 'Qtd. Produtos Cadastrados',
                         marker =  {
                                     'color' : '#EA2027',
                                     'line' : {'width': 1, 'color': '#c0392b'}
                                   },
                         opacity=.9)

# Armazenando gráficos em uma lista
data = [qt_pedidos, qt_produtos]

# Criando Layout
layout = go.Layout(title='Quantidade de Pedidos realizados e Produtos cadastrados por segmento',
                   yaxis={'title':'Produtos cadastrados'},
                   xaxis={'title': 'Segmento'})

# Criando figura que será exibida
fig = go.Figure(data=data, layout=layout)

# Exibindo figura/gráfico
#fig.update_layout(barmode='stack')
fig.show(renderer="colab")

## [VISUALIZAÇÃO] CIDADE

In [222]:
qt_pedidos = go.Bar(x = np.sort(df_pedidos.nm_cidade.unique()),
                    y = df_pedidos.groupby(["nm_cidade"]).count()["id"],
                    name = "Qtd. de Pedidos realizados",
                    marker = {'color': '#b71540'},
                    opacity = .5)

qt_produtos = go.Bar(x = np.sort(df_estabelecimentos.nm_cidade.unique()),
                     y = df_estabelecimentos.groupby("nm_cidade").sum()["qt_produtos_cadastrados"],
                     name = "Qtd. de Produtos cadastrados",
                     marker = {'color': '#e55039'},
                     opacity = .8)

data = [qt_pedidos, qt_produtos]

# Criando Layout
layout = go.Layout(title='Quantidade de Pedidos realizados e Produtos cadastrados por Cidade',
                   yaxis={'title':'Quantidade'},
                   xaxis={'title': 'Cidade'})

# Criando figura que será exibida
fig = go.Figure(data=data, layout=layout)

# Exibindo figura/gráfico
fig.show(renderer="colab")

## [VISUALIZAÇÃO] SISTEMA

In [223]:
qtd_produtos_sistema = go.Bar(x = np.sort(df_estabelecimentos.cd_sistema.unique()),
                              y = df_estabelecimentos.groupby("cd_sistema").sum()["qt_produtos_cadastrados"],
                              name = 'Qtd. Produtos cadastrados',
                              marker = {'color': '#12CBC4'},
                              opacity = 0.2
                             )

qtd_pedidos_sistema = go.Bar(x = np.sort(df_pedidos.cd_sistema.unique()),
                             y = df_pedidos.groupby("cd_sistema").count()["id"],
                             name = 'Qtd. Pedidos realizados',
                             marker = {'color': '#0652DD'},
                             opacity = 0.4
                            )


data = [qtd_produtos_sistema, qtd_pedidos_sistema]

# Criando Layout
layout = go.Layout(title='Quantidade de Pedidos/Produtos por sistema',
                   yaxis={'title':'Quantidade'},
                   xaxis={'title': 'Sistema'})

# Criando figura que será exibida
fig = go.Figure(data=data, layout=layout)

# Exibindo figura/gráfico
fig.show(renderer="colab")


## [VISUALIZAÇÃO] TIPO DE PAGAMENTO

In [7]:

qtd_pedidos = go.Bar(x = np.sort(df_pedidos.tp_pagamento.unique()),
                     y = df_pedidos.groupby(["tp_pagamento"]).count()["id"],
                     name = 'Qtd. Pedidos realizados',
                     marker = {"color": ["#fa983a", "#eb2f06", "#1e3799", "#3c6382"]})

data = [qtd_pedidos]

# Criando Layout
layout = go.Layout(title='Quantidade de Pedidos realizados por Tipo de pagamento',
                   yaxis={'title':'Quantidade'},
                   xaxis={'title': 'Tipo de Pagamento'})

# Criando figura que será exibida
fig = go.Figure(data=data, layout=layout)

# Exibindo figura/gráfico
fig.show(renderer="colab")

## [VISUALIZAÇÃO] MÍDIA

In [8]:

qtd_pedidos = go.Bar(x = np.sort(df_pedidos.nm_canal_midia.unique()),
                     y = df_pedidos.groupby("nm_canal_midia").count()["id"],
                     name = 'Qtd. Pedidos realizados')

qtd_produtos = go.Bar(x = np.sort(df_estabelecimentos.nm_canal_midia.unique()),
                      y = df_estabelecimentos.groupby("nm_canal_midia").sum()["qt_produtos_cadastrados"],
                      name = 'Qtd. Produtos cadastrados')

data = [qtd_pedidos, qtd_produtos]

# Criando Layout
layout = go.Layout(title='Quantidade de Pedidos/Produtos por Mídias',
                   yaxis={'title':'Quantidade'},
                   xaxis={'title': 'Canal de mídia'})

# Criando figura que será exibida
fig = go.Figure(data=data, layout=layout)

# Exibindo figura/gráfico
fig.show(renderer="colab")

## [VISUALIZAÇÃO] VENDEDOR

In [13]:

qtd_pedidos_sistema = go.Bar(x = np.sort(df_pedidos.nm_vendedor.unique()),
                             y = df_pedidos.groupby("nm_vendedor").count()["id"],
                             name = 'Qtd. Pedidos realizados',
                             marker = {"color": "#fad390"})

qtd_produtos_sistema = go.Bar(x = np.sort(df_estabelecimentos.nm_vendedor.unique()),
                              y = df_estabelecimentos.groupby("nm_vendedor").sum()["qt_produtos_cadastrados"],
                              name = 'Qtd. Produtos cadastrados',
                              marker = {"color": "#e55039"})

data = [qtd_pedidos_sistema, qtd_produtos_sistema]

# Criando Layout
layout = go.Layout(title='Quantidade de Pedidos/Produtos por Vendedor',
                   yaxis={'title':'Quantidade'},
                   xaxis={'title': 'Vendedores'})

# Criando figura que será exibida
fig = go.Figure(data=data, layout=layout)

# Exibindo figura/gráfico
fig.show(renderer="colab")